# Report handler examples

Install leakpro as ``` pip install -e /path/to/leakpro ```

### Synthetic examples

In [ ]:
import os
import sys

import pandas as pd

sys.path.append("../..")

from leakpro.synthetic_data_attacks.anomalies import return_anomalies
from leakpro.synthetic_data_attacks.inference_utils import inference_risk_evaluation
from leakpro.synthetic_data_attacks.linkability_utils import linkability_risk_evaluation
from leakpro.synthetic_data_attacks.singling_out_utils import singling_out_risk_evaluation

#Get ori and syn
n_samples = 100
DATA_PATH = "../synthetic_data/datasets/"
ori = pd.read_csv(os.path.join(DATA_PATH, "adults_ori.csv"), nrows=n_samples)
syn = pd.read_csv(os.path.join(DATA_PATH, "adults_syn.csv"), nrows=n_samples)

In [ ]:
syn_anom = return_anomalies(df=syn, n_estimators=1000, n_jobs=-1, verbose=True)
print("Syn anom shape",syn_anom.shape)

In [ ]:
# Create a singling-out result
sin_out_res = singling_out_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    n_attacks = syn_anom.shape[0]
)

In [ ]:
# Create linkability result
link_res = linkability_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    n_samples = syn_anom.shape[0],
    n_attacks = 100
)

In [ ]:
# # Create base-case inference result
inf_res = inference_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    worst_case_flag = False,
    n_attacks = syn_anom.shape[0]
)

# # Create worst-case inference result
inf_res_worst = inference_risk_evaluation(
    dataset = "adults",
    ori = ori,
    syn = syn_anom,
    worst_case_flag = True,
    n_attacks = syn_anom.shape[0]
)

### Gradient inversion example

In [ ]:
sys.path.append("../gia/cifar10_inverting_1_image/")
from cifar import get_cifar10_loader
from model import ResNet
from torchvision.models.resnet import BasicBlock

from leakpro.attacks.gia_attacks.invertinggradients import InvertingConfig
from leakpro.fl_utils.gia_train import train
from leakpro.run import run_inverting

model = ResNet(BasicBlock, [5, 5, 5], num_classes=10, base_width=16 * 10)
client_dataloader, data_mean, data_std = get_cifar10_loader(num_images=1, batch_size=1, num_workers=2)

# Meta train function designed to work with GIA
train_fn = train

# Baseline config
configs = InvertingConfig()
configs.at_iterations = 80 # Decreased from 8000 to avoid GPU memory crash

name = "my_gia_results"
GIA_result = run_inverting(model, client_dataloader, train_fn, data_mean, data_std, configs, experiment_name=name, save=True)

### Membership Inference Attack, CIFAR example

In [ ]:
import os
import sys
import yaml

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(project_root)

In [ ]:
from mia_utils.utils.cifar_data_preparation import get_cifar_dataloader
from mia_utils.utils.cifar_model_preparation import ResNet18, create_trained_model_and_metadata


# Load the config.yaml file
with open('mia_utils/train_config.yaml', 'r') as file:
    train_config = yaml.safe_load(file)

# Generate the dataset and dataloaders
path = os.path.join(os.getcwd(), train_config["data"]["data_dir"])

train_loader, test_loader = get_cifar_dataloader(path, train_config)

In [ ]:
# Train the model
if not os.path.exists("target"):
    os.makedirs("target")
if train_config["data"]["dataset"] == "cifar10":
    num_classes = 10
elif train_config["data"]["dataset"] == "cifar100":
    num_classes = 100
else:
    raise ValueError("Invalid dataset name")

model = ResNet18(num_classes = num_classes)
train_acc, train_loss, test_acc, test_loss = create_trained_model_and_metadata(model, 
                                                                               train_loader, 
                                                                               test_loader, 
                                                                               train_config)

##### Run the MIA attack

In [ ]:
from mia_utils.cifar_handler import CifarInputHandler

from leakpro import LeakPro

# Read the config file
config_path = "mia_utils/audit.yaml"

# Prepare leakpro object
leakpro = LeakPro(CifarInputHandler, config_path)

# Run the audit 
mia_results = leakpro.run_audit(return_results=True)

2024-12-11 13:30:05,252 INFO     Target model blueprint created from ResNet18 in ./mia_utils/utils/cifar_model_preparation.py.
2024-12-11 13:30:05,254 INFO     Loaded target model metadata from ./target/model_metadata.pkl


{'audit': {'random_seed': 1234, 'attack_list': {'rmia': {'training_data_fraction': 0.5, 'attack_data_fraction': 0.5, 'num_shadow_models': 3, 'online': True, 'temperature': 2, 'gamma': 2.0, 'offline_a': 0.33, 'offline_b': 0.66}, 'population': {'attack_data_fraction': 1.0}, 'lira': {'training_data_fraction': 0.5, 'num_shadow_models': 3, 'online': True}, 'loss_traj': {'training_distill_data_fraction': 0.7, 'number_of_traj': 10, 'label_only': False, 'mia_classifier_epochs': 100}, 'HSJ': {'attack_data_fraction': 0.01, 'target_metadata_path': './target/model_metadata.pkl', 'num_iterations': 2, 'initial_num_evals': 100, 'max_num_evals': 10000, 'stepsize_search': 'geometric_progression', 'gamma': 1.0, 'constraint': 2, 'batch_size': 50, 'verbose': True, 'epsilon_threshold': '1e-6'}}, 'output_dir': './leakpro_output', 'attack_type': 'mia', 'modality': 'image'}, 'target': {'module_path': './mia_utils/utils/cifar_model_preparation.py', 'model_class': 'ResNet18', 'target_folder': './target', 'data_

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
2024-12-11 13:30:05,518 INFO     Loaded target model from ./target
2024-12-11 13:30:06,417 INFO     Loaded population dataset from ./data/cifar10.pkl
2024-12-11 13:30:06,419 INFO     Loaded population dataset from ./data/cifar10.pkl
2024-12-11 13:30:06,420 INFO     Creating shadow model handler singleton
2024-12-11 13:30:06,423 INFO     Creating distillation model handler singleton
2024-12-11 13:30:06,425 INFO     Configuring RMIA attack
2024-12-11 13:30:06,426 INFO     Added attack: rm

In [ ]:
# Import and initialize ReportHandler
from leakpro.reporting.report_handler import ReportHandler

# Set report_dir to "./leakpro_output/results" to the results to a local results folder
#    or don't use the report_dir argument to let the ReportHandler find an already
#    existing results folder
report_handler = ReportHandler(report_dir="./leakpro_output/results")

# Save Synthetic results using the ReportHandler
report_handler.save_results(attack_name="singling_out", result_data=sin_out_res)
report_handler.save_results(attack_name="linkability_risk", result_data=link_res)
report_handler.save_results(attack_name="inference_risk_base", result_data=inf_res)
report_handler.save_results(attack_name="inference_risk_worst", result_data=inf_res_worst)

# Save GIA results using report handler
report_handler.save_results(attack_name="gia", result_data=GIA_result)

# Save MIA resuls using report handler
for res in mia_results:
    report_handler.save_results(attack_name=res.attack_name, result_data=res, config=res.configs)

2024-12-11 13:41:35,400 INFO     Initializing report handler...
2024-12-11 13:41:35,402 INFO     report_dir set to: ./leakpro_output/results
2024-12-11 13:41:35,403 INFO     Saving results for rmia
2024-12-11 13:41:40,807 INFO     Saving results for population
2024-12-11 13:41:45,987 INFO     Saving results for lira
2024-12-11 13:41:51,253 INFO     Saving results for loss_traj
2024-12-11 13:41:56,788 INFO     Saving results for HSJ


<Figure size 640x480 with 0 Axes>

In [3]:
# Use the ReportHandler and load all the saved results
report_handler.load_results()

# Create results and collect corresponding latex texts
report_handler.create_results_all()

# Create the report by compiling the latex text
report_handler.create_report()

2024-12-11 13:45:59,879 INFO     PDF compiled


<Figure size 640x480 with 0 Axes>

<Figure size 1100x400 with 0 Axes>

<Figure size 1100x700 with 0 Axes>

<Figure size 1100x700 with 0 Axes>

<Figure size 1100x400 with 0 Axes>